In [ ]:
#mirror whole ct
import SimpleITK as sitk

img = sitk.ReadImage(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii")

flipped_array = sitk.GetArrayFromImage(img)[:, :, ::-1]  
img_flipped = sitk.GetImageFromArray(flipped_array)
img_flipped.CopyInformation(img) 

sitk.WriteImage(img_flipped, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_mirrored.nii")
print("saved")


In [ ]:
#mirror mandible
import SimpleITK as sitk

img = sitk.ReadImage(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\TotalSegmentator1\mandible.nii.gz")

flipped_array = sitk.GetArrayFromImage(img)[:, :, ::-1]  
img_flipped = sitk.GetImageFromArray(flipped_array)
img_flipped.CopyInformation(img) 

sitk.WriteImage(img_flipped, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\TotalSegmentator1\mandible_mirrored.nii.gz")
print("saved")


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# ---------------------- User paths ----------------------
ct1_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii"
ct2_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_mirrored.nii"
# ---------------------------------------------------------

# ---- Load CTs ----
ct1_img = nib.load(ct1_path)
ct2_img = nib.load(ct2_path)

ct1 = ct1_img.get_fdata()
ct2 = ct2_img.get_fdata()

# Normalise for viewing
ct1 = (ct1 - np.min(ct1)) / (np.max(ct1) - np.min(ct1))
ct2 = (ct2 - np.min(ct2)) / (np.max(ct2) - np.min(ct2))

spacing = ct1_img.header.get_zooms()

# Colormap for CT2 overlay
overlay_cmap = plt.cm.bone   # pink/yellow-ish
overlay_cmap.set_under(alpha=0)  # invisible for low values


# ---------------------- Slice Viewer ----------------------
def show_slice(slice_index, plane, alpha=0.35):

    plt.figure(figsize=(6, 6))

    # ---------------- Axial ----------------
    if plane == 'Axial':
        img1 = np.rot90(ct1[:, :, slice_index])
        img2 = np.rot90(ct2[:, :, slice_index])
        aspect = spacing[1] / spacing[0]

    # ---------------- Coronal ----------------
    elif plane == 'Coronal':
        img1 = np.flipud(np.rot90(ct1[:, slice_index, :]))
        img2 = np.flipud(np.rot90(ct2[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]

    # ---------------- Sagittal ----------------
    elif plane == 'Sagittal':
        img1 = np.flipud(np.rot90(ct1[slice_index, :, :]))
        img2 = np.flipud(np.rot90(ct2[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]

    # ---- Base CT ----
    plt.imshow(img1, cmap='gray', origin='lower', aspect=aspect)

    # ---- Overlay CT ----
    overlay = overlay_cmap(img2)
    overlay[..., 3] = alpha   # set transparency
    plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")

    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


# ---------------------- Widgets ----------------------
plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=ct1.shape[2] // 2,
    min=0,
    max=ct1.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Overlay opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


# update slice range when plane changes
def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = ct1.shape[2] - 1
        slice_slider.value = ct1.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = ct1.shape[1] - 1
        slice_slider.value = ct1.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = ct1.shape[0] - 1
        slice_slider.value = ct1.shape[0] // 2

plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(
    show_slice,
    slice_index=slice_slider,
    plane=plane_dropdown,
    alpha=alpha_slider
)


In [ ]:
import SimpleITK as sitk

fixed = sitk.ReadImage(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii")   
moving = sitk.ReadImage(r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_mirrored.nii")   

initial_tx = sitk.CenteredTransformInitializer(
    fixed,
    moving,
    sitk.VersorRigid3DTransform(),
    sitk.CenteredTransformInitializerFilter.GEOMETRY
)

reg = sitk.ImageRegistrationMethod()

reg.SetMetricAsMattesMutualInformation(numberOfHistogramBins=50)
reg.SetMetricSamplingPercentage(0.2, sitk.sitkWallClock)
reg.SetMetricSamplingStrategy(reg.RANDOM)

reg.SetInterpolator(sitk.sitkLinear)

reg.SetOptimizerAsRegularStepGradientDescent(
    learningRate=2.0,
    minStep=1e-4,
    numberOfIterations=200,
    gradientMagnitudeTolerance=1e-6
)

# prevents big rotations that cause the optimizer to flip
reg.SetOptimizerScalesFromPhysicalShift()

reg.SetInitialTransform(initial_tx, inPlace=False)

final_tx = reg.Execute(fixed, moving)

sitk.WriteTransform(final_tx, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\rigid_mirror_to_original.tfm")
print("Saved registration transform")

resampler = sitk.ResampleImageFilter()

resampler.SetReferenceImage(fixed)
resampler.SetInterpolator(sitk.sitkLinear)
resampler.SetTransform(final_tx)

registered_mirror_CT = resampler.Execute(moving)

sitk.WriteImage(registered_mirror_CT, r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror_registered_ct.nii.gz")
print("Saved registered CT")


In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import os

# ---------------------- User paths ----------------------
ct1_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\1720903_plan_shifted.nii"
ct2_path = r"Z:\Angie\SMILE_facialdeformation\StJude_cohort\abby\1720903\Plan\mirror_registered_ct.nii.gz"
# ---------------------------------------------------------

# ---- Load CTs ----
ct1_img = nib.load(ct1_path)
ct2_img = nib.load(ct2_path)

ct1 = ct1_img.get_fdata()
ct2 = ct2_img.get_fdata()

# Normalise for viewing
ct1 = (ct1 - np.min(ct1)) / (np.max(ct1) - np.min(ct1))
ct2 = (ct2 - np.min(ct2)) / (np.max(ct2) - np.min(ct2))

spacing = ct1_img.header.get_zooms()

# Colormap for CT2 overlay
overlay_cmap = plt.cm.bone   # pink/yellow-ish
overlay_cmap.set_under(alpha=0)  # invisible for low values


# ---------------------- Slice Viewer ----------------------
def show_slice(slice_index, plane, alpha=0.35):

    plt.figure(figsize=(6, 6))

    # ---------------- Axial ----------------
    if plane == 'Axial':
        img1 = np.rot90(ct1[:, :, slice_index])
        img2 = np.rot90(ct2[:, :, slice_index])
        aspect = spacing[1] / spacing[0]

    # ---------------- Coronal ----------------
    elif plane == 'Coronal':
        img1 = np.flipud(np.rot90(ct1[:, slice_index, :]))
        img2 = np.flipud(np.rot90(ct2[:, slice_index, :]))
        aspect = spacing[2] / spacing[0]

    # ---------------- Sagittal ----------------
    elif plane == 'Sagittal':
        img1 = np.flipud(np.rot90(ct1[slice_index, :, :]))
        img2 = np.flipud(np.rot90(ct2[slice_index, :, :]))
        aspect = spacing[2] / spacing[1]

    # ---- Base CT ----
    plt.imshow(img1, cmap='gray', origin='lower', aspect=aspect)

    # ---- Overlay CT ----
    overlay = overlay_cmap(img2)
    overlay[..., 3] = alpha   # set transparency
    plt.imshow(overlay, origin='lower', aspect=aspect)

    plt.title(f"{plane} slice {slice_index}")
    plt.axis("off")

    clear_output(wait=True)
    display(plt.gcf())
    plt.close()


# ---------------------- Widgets ----------------------
plane_dropdown = widgets.Dropdown(
    options=['Axial', 'Coronal', 'Sagittal'],
    value='Axial',
    description='Plane:'
)

slice_slider = widgets.IntSlider(
    value=ct1.shape[2] // 2,
    min=0,
    max=ct1.shape[2] - 1,
    step=1,
    description='Slice:',
    continuous_update=True,
    layout=widgets.Layout(width='80%')
)

alpha_slider = widgets.FloatSlider(
    value=0.35,
    min=0.0,
    max=1.0,
    step=0.05,
    description='Overlay opacity:',
    continuous_update=True,
    layout=widgets.Layout(width='60%')
)


# update slice range when plane changes
def update_slider_range(*args):
    plane = plane_dropdown.value
    if plane == 'Axial':
        slice_slider.max = ct1.shape[2] - 1
        slice_slider.value = ct1.shape[2] // 2
    elif plane == 'Coronal':
        slice_slider.max = ct1.shape[1] - 1
        slice_slider.value = ct1.shape[1] // 2
    elif plane == 'Sagittal':
        slice_slider.max = ct1.shape[0] - 1
        slice_slider.value = ct1.shape[0] // 2

plane_dropdown.observe(update_slider_range, names='value')
update_slider_range()

widgets.interact(
    show_slice,
    slice_index=slice_slider,
    plane=plane_dropdown,
    alpha=alpha_slider
)
